In [4]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import json

image_path = "/root/GUI-Memory/data/android_in_the_zoo/test_sample/general/GENERAL-4796150115256817663/GENERAL-4796150115256817663_0.png"
def show_image(image_path):
    image = Image.open(image_path)
    display(image)

mode = "flow"
memory = False

tasks = next(os.walk(f"/root/GUI-Memory/CoAT/android-in-the-zoo/intermediate/openai/{mode}"))[1]
print(len(tasks))

20


In [5]:
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import imagesize
# from source.datasets.aitz.action_utils import match_ui_bbox, _resize_annotation_bounding_boxes


def visualize_bboxes(image_path, annotation_abs_positions, point_yx_dict=None, point_radius=15, color="red"):
    # 打开图片
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # 遍历每个边界框并绘制红色方框
    for bbox in annotation_abs_positions:
        # print(bbox)
        y_top_left, x_top_left, box_height, box_width = bbox
        x_bottom_right = x_top_left + box_width
        y_bottom_right = y_top_left + box_height
        draw.rectangle([x_top_left, y_top_left, x_bottom_right, y_bottom_right], outline=color, width=2)

    if point_yx_dict is not None:
        # 绘制橙黄色圆点
        point_y, point_x = point_yx_dict["touch_yx"]
        draw.ellipse(
            (point_x - point_radius, point_y - point_radius, point_x + point_radius, point_y + point_radius),
            fill="green",
            outline="yellow"
        )

        point_y, point_x = point_yx_dict["lift_yx"]
        draw.ellipse(
            (point_x - point_radius, point_y - point_radius, point_x + point_radius, point_y + point_radius),
            fill="purple",
            outline="yellow"
        )

    # 显示图片
    plt.figure(dpi=200)
    plt.imshow(image)
    plt.axis('off')  # 关闭坐标轴
    plt.show()

# 显示原始UI标记，返回对应的ui元素位置列表
def show_step_uis(step, idx=None):
    # 示例用法
    image_path = f"/root/GUI-Memory/data/android_in_the_zoo/test_sample/{step['image_path']}"
    annotation_abs_positions = eval(step["ui_positions"])
    # print(annotation_abs_positions)
    width, height = imagesize.get(image_path)
    result_touch_yx = eval(step['result_touch_yx'])
    y_touch = result_touch_yx[0] * height
    x_touch = result_touch_yx[1] * width    
    result_lift_yx = eval(step['result_lift_yx'])
    y_lift = result_lift_yx[0] * height
    x_lift = result_lift_yx[1] * width

    yx_item = {"touch_yx": (y_touch, x_touch), "lift_yx": (y_lift, x_lift)}
    visualize_bboxes(image_path, annotation_abs_positions, yx_item)

    if idx is not None:
        visualize_bboxes(image_path, [annotation_abs_positions[idx]], color="blue")

In [6]:
target_tasks = ['general-6458194457574888611_0', 'general-6458194457574888611_2', 'web_shopping-3415759859034183184_1', 'web_shopping-3415759859034183184_2', 'install-689623307793919007_0', 'install-689623307793919007_1', 'install-689623307793919007_2', 'install-689623307793919007_3', 'install-689623307793919007_4', 'install-689623307793919007_5', 'install-689623307793919007_6', 'install-689623307793919007_7', 'web_shopping-3572009480102797822_0', 'web_shopping-3572009480102797822_1', 'web_shopping-3572009480102797822_2', 'web_shopping-3572009480102797822_5', 'web_shopping-3572009480102797822_6', 'web_shopping-3572009480102797822_8', 'web_shopping-3572009480102797822_9', 'web_shopping-3572009480102797822_10', 'web_shopping-3572009480102797822_15', 'web_shopping-3572009480102797822_16', 'web_shopping-14640559031231493138_0', 'web_shopping-14640559031231493138_1', 'web_shopping-14640559031231493138_2', 'web_shopping-14640559031231493138_3', 'web_shopping-14640559031231493138_4', 'web_shopping-14640559031231493138_10', 'web_shopping-14640559031231493138_11', 'web_shopping-14640559031231493138_12', 'web_shopping-14640559031231493138_13']

In [ ]:
for task in tasks[:5]:
    print("\n\n\n************\n\n\n")
    
    memory = False
    if memory==False:
        task_dir = f"/root/GUI-Memory/CoAT/android-in-the-zoo/intermediate/openai/flow/{task}"
        print(task_dir)
        task_files = next(os.walk(task_dir))[-1]
        step_ids = list(set([
            int(file.split(".")[0].split("_")[1]) 
            if "web" not in file and "google_apps" not in file
            else int(file.split(".")[0].split("_")[2]) 
            for file in task_files]))
        step_ids.sort()
        region, id = task.split("-")
        test_file = f"/root/GUI-Memory/data/android_in_the_zoo/test_sample/{region.lower()}/{region.upper()}-{id}/{region.upper()}-{id}.json"
        with open(test_file) as f:
            test_data = json.load(f)

        for i, id in enumerate(step_ids):
            if f"{task}_{id}" not in target_tasks:
                continue
            # show observe
            with open(f"{task_dir}/{task}_{id}_0observe.json", "r") as f:
                observe = json.load(f)
            
            # show think_action
            with open(f"{task_dir}/{task}_{id}_1think_action.json", "r") as f:
                think_action = json.load(f)

            # show reflect
            # if os.path.exists(f"{task_dir}/{task}_{i}_2reflect.jsonl"):
            #     with open(f"{task_dir}/{task}_{i}_2reflect.jsonl", "r") as f:
            #         reflect = json.load(f)
                # print(reflect)
            # final action & ground truth
            with open(f"{task_dir}/{task}_{id}.json", "r") as f:
                final_action = json.load(f)
            
            ground_truth = {
                # "result_action_type":test_data[i]["result_action_type"],
                # "result_action_text":test_data[i]["result_action_text"],
                # "result_touch_yx":test_data[i]["result_touch_yx"],
                # "result_lift_yx":test_data[i]["result_lift_yx"],
                "coat_action_desc":test_data[i]["coat_action_desc"],
            }
            # 绘制边框与idx并可视化ground truth
            if final_action['action_predict']['COAT']['tag']['ACTION'] == 'click_element':
                idx = final_action['action_predict']['COAT']['tag']['ARGS']['idx']
                show_step_uis(test_data[i], idx)
            else:
                show_step_uis(test_data[i])
            print(observe)
            print(think_action)
            print(final_action['action_desc'])
            print(ground_truth)




************



/root/GUI-Memory/CoAT/android-in-the-zoo/intermediate/openai/flow/general-6458194457574888611



************



/root/GUI-Memory/CoAT/android-in-the-zoo/intermediate/openai/flow/web_shopping-3415759859034183184



************



/root/GUI-Memory/CoAT/android-in-the-zoo/intermediate/openai/flow/install-689623307793919007



************



/root/GUI-Memory/CoAT/android-in-the-zoo/intermediate/openai/flow/web_shopping-3572009480102797822



************



/root/GUI-Memory/CoAT/android-in-the-zoo/intermediate/openai/flow/web_shopping-14640559031231493138


In [ ]:
task_dir

'web_shopping-14640559031231493138/13'